## This  is a notebook

In [20]:
import pandas as pd
import numpy as np
import zipfile
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_absolute_error

In [21]:
properties = pd.read_csv('../data/properties_2016.csv')
train_data = pd.read_csv('../data/train_2016_v2.csv')
template_submission = pd.read_csv('../data/sample_submission.csv')

C:\Users\x003752\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Preprocessing

- merge data frames
- remove crappy features (missing > .75)
- make sure the types make sense


we have data of houses that have been sold, those have a logerror. This logerror is our train_y. Split data set on logerror / no logerror. The other is test?



In [22]:
merged_df = pd.merge(train_data, properties,on='parcelid')

In [23]:
features = ['bathroomcnt', 
            'bedroomcnt', 
            'longitude', 
            'latitude', 
            'calculatedfinishedsquarefeet', 
            'structuretaxvaluedollarcnt',
            'logerror']

temp_df = merged_df[features].dropna()
properties_filtered = properties[features[:-1]]
properties_filtered = properties_filtered.fillna(properties_filtered.mean())
x_train = temp_df[features[:-1]]
y_train = temp_df.logerror
x_submission = properties_filtered


## Feature engineering

In [24]:
feature_engineering = Imputer() # Just a dummy for now

## Model

In [25]:
model = RandomForestRegressor()

## Pipeline

In [26]:
pipe = Pipeline([#('feature_engineering', feature_engineering),
                 ('model', model)
                 ]
)

In [27]:
# Fit model
pipe.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('model', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [28]:
out_submission = pipe.predict(x_submission)

In [29]:
out_formatted = template_submission
for col in out_formatted.columns.values:
    if col == 'ParcelId':
        pass
    else:
        out_formatted[col] = out_submission
out_formatted.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,-0.09792,-0.09792,-0.09792,-0.09792,-0.09792,-0.09792
1,10759547,-0.13071,-0.13071,-0.13071,-0.13071,-0.13071,-0.13071
2,10843547,0.08121,0.08121,0.08121,0.08121,0.08121,0.08121
3,10859147,-0.00490,-0.00490,-0.00490,-0.00490,-0.00490,-0.00490
4,10879947,-0.06118,-0.06118,-0.06118,-0.06118,-0.06118,-0.06118


In [30]:
template_submission.shape

(2985217, 7)

In [31]:
assert(out_formatted.shape == template_submission.shape)

In [32]:
out_formatted.to_csv('../data/submission_3_JFR', index=False)

In [33]:
out_formatted.to_csv('../data/submission_3_JFR.gz', index=False, compression='gzip')